In [23]:
# figs, width/height
# add regular text (quotes...)

In [ ]:
import os
import pypandoc
import shutil
import re
import glob
from PIL import Image
import pandas as pd
from gtts import gTTS


# WD = "G:/Meine Ablage/Supervision/AMLD-workshop/Slides"
WD = "G:/Meine Ablage/Supervision/UZH-course/Intelligence24/Slides/ "

FIGSDIR = os.path.join(WD, "Figs")
PDFDIR = os.path.join(WD, "pdfs")

# FILENAME = "PerformancePrediction"
FILENAME = "SessionAIEvaluation"

REFLIST = "RefList.csv"
DOCX = os.path.join(WD, f"{FILENAME}.docx")
AUX_FIGS = ["UZH_logo_center.jpg"]
ADD_REFS = False
MAX_REFS = 8
GET_AUDIO = False
TEMPLATE = "./Beamer/doc.txt"

In [25]:
print(os.getcwd())
with open(TEMPLATE, "r") as f:
    doc = f.read()

output = pypandoc.convert_file(
    DOCX,
    "plain",
    outputfile="./Beamer/tmp.txt",
    encoding="utf-8",
    extra_args=("--standalone", "--wrap=none"),
)
with open("./Beamer/tmp.txt", "r") as f:
    text_orig = f.readlines()

g:\Meine Ablage\Software\SlideGenerator


In [26]:
class Slide:

    def __init__(self, title, index):
        self.title = title
        self.items = []
        self.figs = []
        self.refs = None
        self.section = None
        self.spb = None
        self.spa = None
        self.text_after = []
        self.text_before = []
        self.script = []
        self.index = index
        print("%d. %s" % (index, title))

    def add_fig(self, figstring):
        fig = figstring.split(",")

        figdict = {}
        figdict["name"] = fig[0].strip()
        if len(fig) > 1:
            figformat = fig[1].strip()
            if figformat.replace(".", "").isnumeric():
                figdict["W"] = figformat
                figdict["format"] = "page"
            else:
                formatlist = figformat.split("@")
                figdict["format"] = formatlist[0]
                figdict.update(dict([x.split(":") for x in formatlist[1:]]))
        else:
            figdict["format"] = "page"
            figdict["W"] = 0.5

        for l in ["W", "H", "O"]:
            if l in figdict:
                figdict[l] = round(float(figdict[l]), 2)

        self.figs.append(figdict)

    def add_text(self, text):
        if len(self.items) == 0:
            self.text_before.append(text)
        else:
            self.text_after.append(text)

    def add_item(self, item):
        self.items.append(item)

    def add_refs(self, refs):
        self.refs = refs

    def add_section(self, refs):
        self.refs = refs

    def add_space_before(self, spb):
        self.spb = float(spb.strip())

    def add_space_after(self, spa):
        self.spa = float(spa.strip())

    def add_script(self, script):
        self.script.append(script)

    def get_script(self, i, n, add_items=True):
        script_text = ""
        script_text_clean = ""

        header = "%s (%d/%d)" % (self.title, i, n)
        script_text += header + "\n\n"
        script_text += len(header) * "-" + "\n\n"
        if add_items and len(self.items) > 0:
            for item in self.items:
                script_text += "%s\n" % item
                script_text_clean += "%s\n" % item
        if len(self.script) > 0:
            for text in self.script:
                script_text += "%s\n" % text
                script_text_clean += "%s\n" % text

        if script_text != "":
            script_text += 62 * "*" + "\n"
            script_text += 62 * "*" + "\n\n"

        script_text = re.sub(r"\n\s*\n", "\n\n", script_text)
        script_text_clean = re.sub(r"\n\s*\n", "\n\n", script_text_clean)
        script_text_clean += "NEW SLIDE\n"
        return script_text, script_text_clean

    def get_ref_list(self):

        if self.refs is not None:
            return [x.strip() for x in self.refs.split(",")]
        else:
            return []

    def create(self):

        if self.section is not None:
            text = "\section{%s}\n" % self.section
            text += "\\begin{frame}{%s}\n" % self.title
        else:
            text = "\\begin{frame}{%s}\n" % self.title

        if self.spb is not None:
            text += "\\vspace{%.1fcm}\n" % self.spb

        if len(self.text_before) > 0:
            for text_before in self.text_before:
                text += "%s\n\n" % text_before

        if len(self.items) > 0:
            text += "\\begin{itemize}\n"
            for item in self.items:
                text += "\item %s\n" % item
            text += "\end{itemize}\n"

        if len(self.text_after) > 0:
            for text_after in self.text_after:
                text += "%s\n\n" % text_after

        for fig in self.figs:
            figdict = fig
            figname = figdict["name"]
            if figdict["format"] == "portrait":
                text += "\\begin{tikzpicture}[remember picture, overlay]\n"
                text += "\\node[xshift=-1.2cm,yshift=-1.5cm] at (current page.north east){\n"
                text += "\includegraphics[width=1.5cm]{figs/%s}\n" % figname
                text += "};\n"
                if "C" in figdict:
                    text += "\caption{%s}};\n" % figdict["C"]
                text += "\end{tikzpicture}\n"
            elif figdict["format"] == "left":
                text += "\\begin{tikzpicture}[remember picture, overlay]\n"
                text += (
                    "\\node[xshift=%f\\textwidth,yshift=0] at (current page.west){\n"
                    % (figdict["O"])
                )
                text += "\includegraphics[width=%f\\textwidth]{figs/%s}\n" % (
                    figdict["W"],
                    figname,
                )
                text += "};\n"
                text += "\end{tikzpicture}\n"
            elif figdict["format"] == "right":
                text += "\\begin{tikzpicture}[remember picture, overlay]\n"
                text += (
                    "\\node[xshift=-%f\\textwidth,yshift=0] at (current page.east){\n"
                    % (figdict["O"])
                )
                text += "\includegraphics[width=%f\\textwidth]{figs/%s}\n" % (
                    figdict["W"],
                    figname,
                )
                text += "};\n"
                text += "\end{tikzpicture}\n"
            elif "W" in figdict:  # do also height
                text += "\\begin{figure}\n"
                text += "\includegraphics[width=%f\\textwidth]{figs/%s}\n" % (
                    figdict["W"],
                    figname,
                )
                text += "\end{figure}\n"
            else:
                print("Bad format!!")

        if self.spa is not None:
            text += "\\vspace{%.1fcm}\n" % self.spa

        if self.refs is not None:
            text += (
                "\let\\thefootnote\\relax\\footnotetext{\\scriptsize{\\textit{\\textcolor{gray}{%s}}}}\n"
                % self.refs
            )

        text += "\end{frame}\n\n"

        return text

In [27]:
slide = None
slides = []
silence = False
isscript = False
section = None
slide_index = 1
for i, x in enumerate(text_orig):
    if x[0] == "%":
        continue
    if x[0] == "*":
        title = x[2:].strip()
    elif x[0] == "#":
        silence = False
        isscript = False
        # save slide and create a new one
        if slide is not None:
            slides.append(slide)

        slide = Slide(x[2:].split("/")[0].strip(), slide_index)
        slide_index += 1
        if section is not None:
            slide.add_section(section)
            section = None
    elif x[:4] == "END:":
        slides.append(slide)
        break
    elif x[:4] == "Scr:":
        isscript = True
        slide.add_script(x[4:].strip())
    elif isscript:
        slide.add_script(x.strip())
    elif silence:
        continue
    elif x[0] == "+":
        slide.add_text(x[2:].strip())
    elif x[0] == "-":
        slide.add_item(x[2:].strip())
    elif x[:4] == "Fig:":
        slide.add_fig(x[4:].strip())
    elif x[:4] == "Ref:":
        slide.add_refs(x[4:].strip())
    elif x[:4] == "Sec:":
        section = x[4:].strip()
    elif x[:4] == "SPB:":
        slide.add_space_before(x[4:].strip())
    elif x[:4] == "SPA:":
        slide.add_space_after(x[4:].strip())
    elif x[:4] == "Not:":
        silence = True

1. Connectionist Approaches
2. Biological and Artificial Neurons
3. Activation Function
4. Activation Functions
5. Artificial Neural Network (ANN)
6. Layer
7. ANN Intuition
8. ANN Intuition
9. Universal Approximation Theorem
10. Deep Learning
11. Hierarchical Processing
12. Internal Representations
13. Large-Scale Data
14. Technological Progress
15. Layers
16. Graphics and Tensor Processing Units (GPUs and TPUs)
17. Loss
18. ANN Training
19. ANN Training
20. ANN Training
21. Stochastic Optimization
22. Some Common Types of Network
23. Deep Learning Frameworks
24. Pytorch
25. Computational Graph
26. Defining a Simple Network
27. Training Step
28. Exercise


In [28]:
fig_list = []

main_text = ""
script_text = ""
script_text_clean = ""
ref_list = []
refs_text = ""
nslides = len(slides)
for i, slide in enumerate(slides):
    print(i, slide.title)
    main_text += slide.create()
    s, sc = slide.get_script(i + 1, nslides)
    script_text += s
    script_text_clean += sc
    ref_list.extend(slide.get_ref_list())
    fig_list.extend(slide.figs)

0 Connectionist Approaches
1 Biological and Artificial Neurons
2 Activation Function
3 Activation Functions
4 Artificial Neural Network (ANN)
5 Layer
6 ANN Intuition
7 ANN Intuition
8 Universal Approximation Theorem
9 Deep Learning
10 Hierarchical Processing
11 Internal Representations
12 Large-Scale Data
13 Technological Progress
14 Layers
15 Graphics and Tensor Processing Units (GPUs and TPUs)
16 Loss
17 ANN Training
18 ANN Training
19 ANN Training
20 Stochastic Optimization
21 Some Common Types of Network
22 Deep Learning Frameworks
23 Pytorch
24 Computational Graph
25 Defining a Simple Network
26 Training Step
27 Exercise


In [29]:
# Add References
ref_list = list(set(ref_list))
ref_list.sort()

for ref in ref_list:
    refs_text += "%s\n" % ref

if ADD_REFS:
    ref_table = pd.read_csv(os.path.join(WD, REFLIST), delimiter=";")
    print(ref_table)
    ref_table = ref_table.sort_values(by="ref")
    ref_dict = dict(zip(ref_table["ref"], ref_table["title"]))

    os.path.join(WD, REFLIST)
    notfoundrefs = []

    nrefs = 1
    main_text += "\\begin{frame}{References}\n\n"

    for ref in ref_list:
        if ref in ref_dict:
            mytitle = ref_dict[ref]
        else:
            notfoundrefs.append(ref)
            continue
        print(nrefs % MAX_REFS)
        if nrefs % MAX_REFS == 1 and nrefs > 1:
            main_text += "\end{frame}\n\\begin{frame}{References}\n\n"
        nrefs += 1
        refsplit = ref.split(" ")
        author = " ".join(refsplit[:-1])
        print(author)
        year = refsplit[-1]
        if mytitle[-1] not in ["?", ".", "!"]:
            mytitle = mytitle + "."
        main_text += "\small{%s (%s). \\textit{%s}}\n\n" % (author, year, mytitle)
    main_text += "\end{frame}\n"

    print(notfoundrefs)

In [30]:
fig_names = [x["name"] for x in fig_list]
fig_names = fig_names + AUX_FIGS
print(fig_names)
fig_not_found = []

for f in glob.glob("./output/figs/*"):
    os.remove(f)

# convert to jpg

for fig_name in glob.glob(os.path.join(FIGSDIR, "*.png")):
    name, ext = os.path.splitext(fig_name)
    file_jpg = os.path.join(FIGSDIR, name + ".jpg")
    if not os.path.exists(file_jpg):
        print(file_jpg)
        png = Image.open(fig_name).convert("RGBA")
        background = Image.new("RGBA", png.size, (255, 255, 255))
        alpha_composite = Image.alpha_composite(background, png).convert("RGB")
        alpha_composite.save(file_jpg)
        shutil.move(fig_name, os.path.join(FIGSDIR, "./pngs/"))
        # print(fig_name, file_jpg, os.path.join(FIGSDIR, './pngs/'))

for fig_name in fig_names:

    try:
        shutil.copyfile(
            os.path.join(FIGSDIR, fig_name), os.path.join("./output/figs", fig_name)
        )
    except:
        fig_not_found.append(fig_name)

import difflib

fig_list = os.listdir(FIGSDIR)

if len(fig_not_found) > 0:
    print("Figures not found:")
    for ff in fig_not_found:
        print(ff, ":", difflib.get_close_matches(ff, fig_list))
else:
    print("No figures missing!")

['AIMLDL.jpg', 'Neurons.jpg', 'ReLu.jpg', 'Activations.jpg', 'ANNDiagramLabel.jpg', 'LinearLayer.jpg', 'ANN1D-1.jpg', 'ANNProgApprox.jpg', 'DeepLearning.jpg', 'GeoffHinton.jpg', 'VisualHierarchy.jpg', 'FeatureRepresentations.jpg', 'FigImageNet.jpg', 'ComputationalCapacityOverTime.jpg', 'ModelSize.jpg', 'GPU.jpg', 'Optimization.jpg', 'Optimization.jpg', 'ANN1D-0.jpg', 'CNN2.jpg', 'DLFrameworks.jpg', 'NetCode.jpg', 'StepCode.jpg', 'UZH_logo_center.jpg']
No figures missing!


In [31]:
doc = doc.replace("$TITLE", title)
doc = doc.replace("$MAINTEXT", main_text)
with open("./output/main.tex", "w") as f:
    f.write(doc)

with open("./output/script.txt", "w") as f:
    f.write(script_text)

with open("./output/refs.txt", "w") as f:
    f.write(refs_text)

In [32]:
words = script_text_clean.split()
print(f"Total words: {len(words)}")

if GET_AUDIO:
    audio = gTTS(text=script_text_clean, lang="en", slow=False)
    audio.save("./output/script.mp3")
    #    os.system("start ./output/script.mp3")

    from mutagen.mp3 import MP3

    audio = MP3("./output/script.mp3")

    print(f"Total duration: {round(audio.info.length/60, 2)} minutes")
# 60 min -> 7800 sec

Total words: 2614


In [ ]:
# generate pdf
print(os.getcwd())

os.system(f"cd output && pdflatex main.tex")
shutil.copyfile("output/main.pdf", f"./pdfs/{FILENAME}.pdf")
shutil.copyfile("output/main.pdf", f"{PDFDIR}/{FILENAME}.pdf")

g:\Meine Ablage\Software\SlideGenerator


'./pdfs/NeuralNetworks.pdf'